In [43]:
import boto3
import botocore
import csv
import os
from io import StringIO
import psycopg2
from dotenv import load_dotenv #pip install python-dotenv
from psycopg2 import connect, sql
from os import environ as env
import pandas as pd

load_dotenv()
conn_string = os.getenv('conn_string')

if 'conn_string' in env:
    print(env['conn_string'][:35])

dbname='etl_bites' user='joemiller'


## EXTRACT

Download csv file from Amazon S3 bucket.

This S3 bucket is public so we dont need any AWS credentials to access it.

In [44]:
def download_csv_from_s3(bucket_name, object_key):
    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    content = response['Body'].read().decode('utf-8')

    # Process header row to remove leading and trailing spaces
    header, rows = content.split("\n", 1)
    cleaned_header = ",".join(column.strip() for column in header.split(","))

    # Return cleaned content
    return cleaned_header + "\n" + rows

titanic_csv_content = download_csv_from_s3('data-eng-makers-public-datasets-404544469985', 'etl_bites_04_titanic_dataset.csv')

In [45]:
titanic_csv_content

'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked\n1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S\r\n2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C\r\n3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S\r\n4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S\r\n5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S\r\n6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q\r\n7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S\r\n8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S\r\n9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S\r\n10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C\r\n11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S\r\n12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S\r\n13,0,3,"Saunderc

In [46]:
titanic_csv = csv.reader(StringIO(titanic_csv_content))
for row in titanic_csv:
    print(row)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
['1', '0', '3', 'Braund, Mr. Owen Harris', 'male', '22', '1', '0', 'A/5 21171', '7.25', '', 'S']
['2', '1', '1', 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', '38', '1', '0', 'PC 17599', '71.2833', 'C85', 'C']
['3', '1', '3', 'Heikkinen, Miss. Laina', 'female', '26', '0', '0', 'STON/O2. 3101282', '7.925', '', 'S']
['4', '1', '1', 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', '35', '1', '0', '113803', '53.1', 'C123', 'S']
['5', '0', '3', 'Allen, Mr. William Henry', 'male', '35', '0', '0', '373450', '8.05', '', 'S']
['6', '0', '3', 'Moran, Mr. James', 'male', '', '0', '0', '330877', '8.4583', '', 'Q']
['7', '0', '1', 'McCarthy, Mr. Timothy J', 'male', '54', '0', '0', '17463', '51.8625', 'E46', 'S']
['8', '0', '3', 'Palsson, Master. Gosta Leonard', 'male', '2', '3', '1', '349909', '21.075', '', 'S']
['9', '1', '3', 'Johnson, Mrs. Oscar W (Elis

In [47]:
titanic_csv = csv.DictReader(StringIO(titanic_csv_content))
for row in titanic_csv:
    print(row)

{'PassengerId': '1', 'Survived': '0', 'Pclass': '3', 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': '22', 'SibSp': '1', 'Parch': '0', 'Ticket': 'A/5 21171', 'Fare': '7.25', 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': '2', 'Survived': '1', 'Pclass': '1', 'Name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Sex': 'female', 'Age': '38', 'SibSp': '1', 'Parch': '0', 'Ticket': 'PC 17599', 'Fare': '71.2833', 'Cabin': 'C85', 'Embarked': 'C'}
{'PassengerId': '3', 'Survived': '1', 'Pclass': '3', 'Name': 'Heikkinen, Miss. Laina', 'Sex': 'female', 'Age': '26', 'SibSp': '0', 'Parch': '0', 'Ticket': 'STON/O2. 3101282', 'Fare': '7.925', 'Cabin': '', 'Embarked': 'S'}
{'PassengerId': '4', 'Survived': '1', 'Pclass': '1', 'Name': 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Sex': 'female', 'Age': '35', 'SibSp': '1', 'Parch': '0', 'Ticket': '113803', 'Fare': '53.1', 'Cabin': 'C123', 'Embarked': 'S'}
{'PassengerId': '5', 'Survived': '0', 'Pclass': '3', 'Name': 'Allen, Mr. William

## TRANSFORM

Now, let's filter passengers based on a specific passenger class and calculate the average fare for each class:

In [48]:
def calculate_average_fare(titanic_data, pclass_filter):
    total_fare = 0
    passengers_count = 0

    for row in titanic_data:
        if row['Pclass'] == str(pclass_filter):
            total_fare += float(row['Fare'])
            passengers_count += 1

    return total_fare / passengers_count if passengers_count > 0 else 0

average_fare_class_1 = calculate_average_fare(csv.DictReader(StringIO(titanic_csv_content)), 1)

## LOAD

In [49]:
def execute_query_postgresql(conn_string, query):
    with connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

create_api_data_table = '''
DROP TABLE IF EXISTS class_average_fares CASCADE;
CREATE TABLE class_average_fares (
id SERIAL PRIMARY KEY,
pclass INTEGER NOT NULL,
average_fare NUMERIC(10, 2) NOT NULL
);
'''

execute_query_postgresql(conn_string, create_api_data_table)

In [ ]:
def insert_data_to_postgresql(data, connection):
    cursor = connection.cursor()
    query = "INSERT INTO class_average_fares (pclass, average_fare) VALUES (%s, %s)"
    cursor.execute(query, data)
    connection.commit()

conn = psycopg2.connect(conn_string)

insert_data_to_postgresql((1, average_fare_class_1), conn)

In [ ]:
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
cursor.execute("""
       SELECT *
       FROM class_average_fares 
               """
               )
rows = cursor.fetchall()
print(rows)

[(1, 1, Decimal('84.15'))]
